In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from cvxopt import matrix, solvers

In [ ]:
class SVM():    
    def __init__(self, type='Primal', kernel='Gaussian', dim=2, a=0.01, c=0.01, sigma=1):
        self.type = type
        self.dim = dim
        self.kernel = kernel
        self.func = {'Poly':self.poly_kernel, 'Gaussian':self.gaussian_kernel}
        self.a = a
        self.c = c
        self.sigma = sigma
    
    def fit(self, Xtrain, ytrain):
        n = Xtrain.shape[0]
        if self.type == 'Primal':
            Q = np.eye(self.dim + 1)
            Q[0, 0] = 0
            p = np.zeros(self.dim + 1)
            A = -1 * np.multiply(ytrain, np.hstack((np.ones([n, 1]), Xtrain))).T
            c = -1 * np.ones(n)
            Q = matrix(Q.tolist())
            p = matrix(p.tolist())
            A = matrix(A.tolist())
            c = matrix(c.tolist())
            solvers.options['show_progress'] = False
            result = solvers.qp(Q, p, A, c)
            self.b = result['x'][0]
            self.w = result['x'][1:]
            
        elif self.type == 'Dual':
            Q = np.multiply(ytrain, Xtrain)
            Q = Q.dot(Q.T)
            p = -1 * np.ones(n)
            A = -1 * np.eye(n)
            c = np.zeros(n)
            r = ytrain
            Q = matrix(Q.tolist())
            p = matrix(p.tolist())
            A = matrix(A.tolist())
            c = matrix(c.tolist())
            r = matrix(r.tolist())
            v = matrix(0.)
            solvers.options['show_progress'] = False
            result = solvers.qp(Q, p, A, c, r, v)
            result_list = np.array(result['x']).tolist()
            f = lambda x: x if x > 1e-3 else 0
            result_list = [f(*item) for item in result_list]
            index = result_list.index(max(result_list))
            self.w = sum(np.multiply(np.multiply(np.array(result_list).reshape(-1,1), ytrain), Xtrain))
            self.b = ytrain[index] - self.w.dot(Xtrain[index, :].T)
            
        elif self.type == 'Kernel':
            Q1 = ytrain.dot(ytrain.T)
            Q2 = np.zeros([n, n])
            for i in range(n):
                for j in range(n):
                    Q2[i ,j] = self.func[self.kernel](Xtrain[i, :].T, Xtrain[j, :].T)
            Q = np.multiply(Q1, Q2)
            p = -1 * np.ones(n)
            A = -1 * np.eye(n)
            c = np.zeros(n)
            r = ytrain
            Q = matrix(Q.tolist())
            p = matrix(p.tolist())
            A = matrix(A.tolist())
            c = matrix(c.tolist())
            r = matrix(r.tolist())
            v = matrix(0.)
            solvers.options['show_progress'] = False
            result = solvers.qp(Q, p, A, c, r, v)
            alpha = np.array(result['x']).tolist()
            f = lambda x: x if x > 1e-3 else 0
            alpha = np.array([f(*item) for item in alpha]).reshape(-1,1)
            self.index = (alpha > 0).ravel()
            self.coef = np.multiply(alpha[self.index], ytrain[self.index])
            self.xn = Xtrain[self.index, :]
            self.b = 0
            index = self.index.tolist().index(True)
            self.b = ytrain[index] - self.distance(Xtrain[index, :].T)
                
        else : 
            print('未知算法')
            return
            
    def score(self, Xtest, ytest):
        n = ytest.shape[0]
        count = 0
        for i in range(n):
            if self.predict(Xtest[i, :].T) != ytest[i]:
                count = count + 1 
        print(self.type + ' SVM正确率为' + str((n-count)/n)[:6])
        return (n-count)/n
    
    def predict(self, x):
        return np.sign(self.distance(x))
        
    def distance(self, X):
        if self.type == 'Kernel':
            n = self.xn.shape[0]
            if len(X.shape) == 1:
                result = np.zeros(n)
                for i in range(n):
                    result[i] = self.coef[i] * self.func[self.kernel](self.xn[i, :].T, X.T)
                return sum(result) + self.b
            elif len(X.shape) == 2:
                m = X.shape[0]
                result = np.zeros([n, m])
                for i in range(n):
                    for j in range(m):
                        result[i, j] = self.coef[i] * self.func[self.kernel](self.xn[i, :].T, X[j, :].T)
                return sum(result) + self.b
            else:
                print('维度错误')
                return
        else :         
            return X.dot(self.w) + self.b
    
    def poly_kernel(self, x, y):
        return (self.a * x.T.dot(y) + self.c)**4
        
    def gaussian_kernel(self, x, y):
        return np.exp(-(x - y).T.dot(x - y)/(2 * self.sigma**2))
    
    def plot(self, Xtrain, ytrain, Xtest, ytest):
        train_index = (ytrain==1).ravel()
        test_index = (ytest==1).ravel()
        plt.figure()
        plt.scatter(Xtrain[train_index, 0], Xtrain[train_index, 1], marker='.', c='b', s=100
                    , label='train +1')
        plt.scatter(Xtrain[~train_index, 0], Xtrain[~train_index, 1], marker='.', c='k', s=100
                   , label='train -1')
        plt.scatter(Xtest[test_index, 0], Xtest[test_index, 1], marker='D', c='r', label='test +1')
        plt.scatter(Xtest[~test_index, 0], Xtest[~test_index, 1], marker='D', c='y', label='test -1')
        ax = plt.gca()
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        axisx = np.linspace(xlim[0], xlim[1], 30)
        axisy = np.linspace(ylim[0], ylim[1], 30)
        axisx, axisy = np.meshgrid(axisx, axisy)
        xy = np.vstack([axisx.ravel(), axisy.ravel()]).T
        z = self.distance(xy).reshape(axisx.shape)
        plt.contour(axisx, axisy, z, levels=[-1, 0, 1], linestyles=['--', '-', '--'], alpha=1)
        plt.legend()
        if self.type == 'Kernel':
            plt.title(self.type + ' SVM' + ' with ' + self.kernel + ' Kernel')
        else :
            plt.title(self.type + ' SVM')

In [ ]:
mean1 = [-5, 0]
s1 = [[1, 0], [0, 1]]
data1 = np.random.multivariate_normal(mean1, s1, 200)
mean2 = [0, 5]
s2 = [[1, 0], [0, 1]]
data2 = np.random.multivariate_normal(mean2, s2, 200)
X = np.vstack((data1, data2))
ones = np.ones(200).reshape(-1,1)
y = np.vstack((ones,-1*ones))
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [ ]:
model1 = SVM('Primal')
model1.fit(Xtrain, ytrain)
model1.score(Xtrain,ytrain)
model1.score(Xtest, ytest);
model1.plot(Xtrain, ytrain, Xtest, ytest)

In [ ]:
model2 = SVM('Dual')
model2.fit(Xtrain, ytrain)
model2.score(Xtrain,ytrain)
model2.score(Xtest, ytest);
model2.plot(Xtrain, ytrain, Xtest, ytest)

In [ ]:
model3 = SVM('Kernel', kernel='Poly')
model3.fit(Xtrain, ytrain)
model3.score(Xtrain,ytrain)
model3.score(Xtest, ytest);
model3.plot(Xtrain, ytrain, Xtest, ytest)

In [ ]:
model4 = SVM('Kernel', kernel='Gaussian', sigma=10)
model4.fit(Xtrain, ytrain)
model4.score(Xtrain,ytrain)
model4.score(Xtest, ytest);
model4.plot(Xtrain, ytrain, Xtest, ytest)

In [ ]:
def dataset(type):
    if  type == '沿海':
        x1 = np.array([[119.28,26.08],
             [121.31,25.03],
             [121.47,31.23],
             [118.06,24.27],
             [121.46,39.04],
             [122.10,37.50],
             [124.23,40.07]])
        y1 = np.ones((7, 1))
        x2 = np.array([[129.87,32.75],
             [130.33,31.36],
             [131.42,31.91],
             [130.24,33.35],
             [133.33,15.43],
             [138.38,34.98],
             [140.47,36.37]])
        y2 = -1 * np.ones((7, 1))
    elif type == '内陆':
        x1 = np.array([[119.28,26.08],
             [121.31,25.03],
             [121.47,31.23],
             [118.06,24.27],
             [113.53,29.58],
             [104.06,30.67],
             [116.25,39.54],
             [121.46,39.04],
             [122.10,37.50],
             [124.23,40.07]])
        y1 = np.ones((10, 1))
        x2=np.array([[129.87,32.75],
             [130.33,31.36],
             [131.42,31.91],
             [130.24,33.35],
             [136.54,35.10],
             [132.27,34.24],
             [139.46,35.42],
             [133.33,15.43],
             [138.38,34.98],
             [140.47,36.37]])
        y2 = -1 * np.ones((10, 1))
    return np.vstack((x1, x2)), np.vstack((y1, y2))  

Xtrain, ytrain = dataset('内陆')
xc = np.array([123.28,25.45])

In [ ]:
model = SVM('Kernel', kernel='Gaussian', sigma=10)
model.fit(Xtrain, ytrain)
if model.predict(xc) == 1:
    print('中国')
else :
    print('日本')